# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [4]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [5]:
spark = SparkSession.builder.appName('linregproj').getOrCreate()

In [6]:
# Get data
data = spark.read.csv('cruise_ship_info.csv',
                      header=True,inferSchema=True)

In [7]:
data.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [10]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [34]:
# Check if any string data can be categorical
data.groupBy('Cruise_line').count().show()

# Cruise line can affect the crew of the ship!

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [35]:
from pyspark.ml.feature import StringIndexer

In [36]:
# Transform string data into a category
indexer = StringIndexer(inputCol='Cruise_line',
                        outputCol='cruise_cat')
indexed = indexer.fit(data).transform(data)

In [39]:
indexed.head(3) # Notice the cruise_cat

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0),
 Row(Ship_name='Celebration', Cruise_line='Carnival', Age=26, Tonnage=47.262, passengers=14.86, length=7.22, cabins=7.43, passenger_density=31.8, crew=6.7, cruise_cat=1.0)]

In [11]:
# Setup the dataframe for Machine Learning
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [40]:
# We only want numerical data
# Predict crew, based on all other features
indexed.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [41]:
# Create a vector assembler with the features you want
assembler = VectorAssembler(inputCols=['Age',
                                       'Tonnage',
                                       'passengers',
                                       'length',
                                       'cabins',
                                       'passenger_density',
                                       'cruise_cat'],
                            outputCol='features')

In [42]:
# Pass your data through the assembler to transform it
output = assembler.transform(indexed)

In [43]:
output.printSchema() # new features column!

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_cat: double (nullable = false)
 |-- features: vector (nullable = true)



In [44]:
output.head(1)

[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0, features=DenseVector([6.0, 30.277, 6.94, 5.94, 3.55, 42.64, 16.0]))]

In [45]:
# Final dataframe for Machine Learning
final_data = output.select('features','crew')

In [46]:
final_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [47]:
# Now, split the data
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [48]:
# Double-Check the split
train_data.describe().show()
test_data.describe().show()

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|               120|
|   mean| 7.936083333333344|
| stddev|3.6571213586170015|
|    min|              0.59|
|    max|              21.0|
+-------+------------------+

+-------+------------------+
|summary|              crew|
+-------+------------------+
|  count|                38|
|   mean|7.3460526315789485|
| stddev| 2.965870310694275|
|    min|              0.59|
|    max|             12.53|
+-------+------------------+



In [49]:
# Create a LinReg model, fit to train_data and test on test_data
lr = LinearRegression(featuresCol='features',
                      labelCol='crew')

In [50]:
lrModel = lr.fit(train_data)

In [51]:
test_results = lrModel.evaluate(test_data)

In [52]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -1.4314968035488924|
|  0.2986384633952195|
| -0.6034868812475676|
|  1.0738424994860694|
|-0.07026804378552498|
| -0.4502069298785951|
| 0.03165136399239987|
|-0.49855027389113893|
|-0.26406699906614506|
|  1.8124132412357081|
|-0.38107873442164575|
|-0.07737340686004845|
|-0.01102839064450...|
| -1.2166991387300072|
|0.009414073551113677|
|  0.8232156300161773|
|  0.3121346822873985|
| -1.1295893231321914|
|   0.379341963841183|
|  -1.813604169066119|
+--------------------+
only showing top 20 rows



In [53]:
test_results.r2

0.9370911934527716

In [54]:
test_results.rootMeanSquaredError

0.7340352960284293

In [55]:
final_data.describe().show()
# We have a variation of roughly 0.6 of a crew member
# on a mean of 7.7 crew members. Good!

# WARNING! I HAVE CHECKED PREVIOUSLY
# IF YOU REMOVE THE CRUISE CATEGORY, THE MODEL IMPROVES!!!

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [58]:
# View correlation
from pyspark.sql.functions import corr

In [61]:
data.select(corr('crew','passengers')).show()
data.select(corr('crew','cabins')).show()

+----------------------+
|corr(crew, passengers)|
+----------------------+
|    0.9152341306065384|
+----------------------+

+------------------+
|corr(crew, cabins)|
+------------------+
|0.9508226063578497|
+------------------+

